In [ ]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version은 코랩 명령입니다.
    %tensorflow_version 2.x
    !pip install -q -U tfx
    print("패키지 호환 에러는 무시해도 괜찮습니다.")
except Exception:
    pass

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "data"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# 13.1 데이터 API

In [ ]:
X = tf.range(10)

for i in X:
  print(X)


from_tensor_slices()

X의 각 원소가 아이템으로 표현됨


In [ ]:
X= tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)

for i in dataset:
  print(i)

### 13.1.1 연쇄 변환

In [ ]:
dataset = dataset.repeat(3).batch(7)
#세번 반복되는 데이터셋으로 바꾼 후 7개씩 묶는 연쇄변환

In [ ]:
for i in dataset:
  print(i)

map 가능 : 각 아이템에 변환

In [ ]:
dataset = dataset.map(lambda x : x+2)

In [ ]:
for i in dataset:
  print(i)

apply 가능: 전체 데이터셋을 변환

In [ ]:
dataset = dataset.apply(tf.data.Dataset.unbatch)

In [ ]:
for i in dataset:
  print(i)

In [ ]:
dataset = dataset.filter(lambda x: x<10)

In [ ]:
for i in dataset:
  print(i)

### 13.1.2 데이터 셔플링

In [ ]:
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5,seed=42).batch(7)

In [ ]:
for i in dataset:
  print(i)

여러 파일에서 한줄씩 번갈아 읽기 (진짜 셔플을 위해)

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1,1), random_state=42    
)

X_train,X_valid, y_train,y_valid = train_test_split(
    X_train_full,y_train_full,random_state=42
)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

메모리에 맞지 않는 매우 큰 데이터셋인 경우 일반적으로 먼저 여러 개의 파일로 나누고 텐서플로에서 이 파일들을 병렬로 읽게함.

 데모를 위해 주택 데이터셋을 20개의 CSV 파일 분할

In [ ]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.csv")

    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

In [ ]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

train_filepaths = save_to_multiple_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, "test", header, n_parts=10)

In [ ]:
import pandas as pd
pd.read_csv(train_filepaths[0]).head()

In [ ]:
with open(train_filepaths[0]) as f:
  for i in range(5):
    print(f.readline(), end="")

In [ ]:
train_filepaths

입력 파이프라인 만들기

list_files()

In [ ]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths,seed=42)

In [ ]:
#호출할때마다 섞어서 반환됨
for filepath in filepath_dataset:
  print(filepath)

interleave()

In [ ]:
n_readers= 5

#interleave는 filepath_dataset에 있는 파일 경로 다섯개를 번갈아가면서 읽음.
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length = n_readers
)

In [ ]:
for line in dataset.take(5):
  print(line.numpy())

### 13.1.3 데이터 전처리

tf.io.decode_csv 의 디폴트값 지정 방법

In [ ]:
#디폴트값 지정
record_defaults=[0, np.nan, tf.constant(np.nan, dtype=tf.float64), "Hello", tf.constant([])]
#1,2,3,4,5를 각 특성이 어떻게 해석하는지
parsed_fields = tf.io.decode_csv('1,2,3,4,5', record_defaults)

parsed_fields

In [ ]:
#, , , , 5를 어떻게 해석하는지
parsed_fields = tf.io.decode_csv(',,,,5', record_defaults)
parsed_fields

In [ ]:
#다른 값들과 다르게 다섯번째 필드는 기본값을 지정하지 않음 --> tf.constant([])
#따라서 ,,,,로값을 전달하지 않으면 예외가 발생함. 
try:
    parsed_fields = tf.io.decode_csv(',,,,', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

In [ ]:
#필드 개수는 record_defaults의 필드 수와도 맞아야함.
try:
    parsed_fields = tf.io.decode_csv('1,2,3,4,5,6,7', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

전처리를 위한 간단한 함수 만들어보기

In [ ]:
n_inputs = 8 

def preprocess(line):
  defs = [0.] * n_inputs +[tf.constant([], dtype = tf.float32)]
  fields = tf.io.decode_csv(line, record_defaults=defs)
  x= tf.stack(fields[:-1])
  y =tf.stack(fields[-1:])
  return (x-X_mean)/X_std , y

In [ ]:
preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')

### 13.1.4 데이터 적재와 전처리를 합치기

앞서 배운 적재, 전처리, 셔플링, 반복, 배치를 적용

In [ ]:
def csv_reader_dataset(filepaths, repeat=1,n_readers=5,
                       n_read_threads=None, shuffle_buffer_size =10000,
                       n_parse_threads=5, batch_size=32):
  dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
  #파일경로를 섞은 데이터셋 반환
  dataset = dataset.interleave(
      lambda filepath : tf.data.TextLineDataset(filepath).skip(1),
      cycle_length = n_readers, num_parallel_calls = n_read_threads
  )
  #번갈아가며 읽기. 
  #cycle_length 몇개파일을 번갈아가면 읽을지 , num_parallel_calls 병렬처리
  
  dataset = dataset.shuffle(shuffle_buffer_size)
  #interleave 후 shuffle --> 완벽? 한 셔플
  dataset = dataset.map(preprocess, num_parallel_calls = n_parse_threads)
  dataset = dataset.batch(batch_size)
  return dataset.prefetch(1)


In [ ]:
tf.random.set_seed(42)

train_set = csv_reader_dataset(train_filepaths, batch_size=3)
for X_batch,y_batch in train_set.take(2):
  print("X = ",X_batch)
  print("y = ",y_batch)
  print()

### 13.1.6 tf.keras와 데이터셋 사용하기

In [ ]:
train_set = csv_reader_dataset(train_filepaths, repeat=None) #데이터셋 무한반복
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

In [ ]:
model = keras.models.Sequential([
                                keras.layers.Dense(30,activation='relu',
                                                   input_shape=X_train.shape[1:]),
                                keras.layers.Dense(1)
])

In [ ]:
model.compile(loss= keras.losses.MeanSquaredError(), optimizer = keras.optimizers.SGD(lr=1e-3))

In [ ]:
batch_size=32
model.fit(train_set, steps_per_epoch=len(X_train) // batch_size, epochs=10,
          validation_data=valid_set)

evaluate 가능

In [ ]:
model.evaluate(test_set, steps = len(X_test)//batch_size)

predict 가눙~

In [ ]:
new_set = test_set.map(lambda X,y : X) #레이블 없애기
X_new = X_test
model.predict(new_set, steps= len(X_new)//batch_size)

사용자 정의 훈련반복도 가능~

In [ ]:
n_epochs=5
batch_size=32
n_steps_per_epochs = len(X_train)//batch_size
total_steps = n_epochs * n_steps_per_epochs
#train_set 이 지금 무한 repeat이기 때문에 끝을 직접 계산해서 정해줘야함
global_steps =0

optimizer = keras.optimizers.Nadam(lr=0.01)
loss_fn = keras.losses.mean_squared_error




for X_batch, y_batch in train_set.take(total_steps):
  global_steps +=1
  print("\rGlobal step {}/{}".format(global_steps,total_steps), end = "")

  with tf.GradientTape() as tape:
    y_pred = model(X_batch)
    main_loss = tf.reduce_mean(loss_fn(y_pred,y_batch))
    loss = tf.add_n([main_loss] + model.losses)
  gradient = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradient, model.trainable_variables))


전체 훈련을 반복하는 텐서플로 함수 만들어보기

In [ ]:
optimizer = keras.optimizers.Nadam(lr = 0.01)
loss_fn = keras.losses.mean_squared_error

@tf.function
def train(model, n_epochs, batch_size= 32,
          n_readers=5, n_read_threads=5, shuffle_buffer_size=10000,
          n_parse_threads=5):
  train_set =csv_reader_dataset(train_filepaths,repeat=n_epochs, 
                                n_readers=n_readers,
                                n_read_threads = n_read_threads, 
                                shuffle_buffer_size=shuffle_buffer_size,
                                n_parse_threads= n_parse_threads, 
                                batch_size=batch_size)
  global_steps=0
  for X_batch,y_batch in train_set:
    global_steps+=1
    if tf.equal(global_steps % 100,0):
      tf.print("\rGlobal step", global_steps, "/", total_steps)
    with tf.GradientTape() as tape:
      y_pred = model(X_batch)
      main_loss = tf.reduce_mean(loss_fn(y_pred,y_batch))
      loss = tf.add_n([main_loss] + model.losses)

    gradient = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient,model.trainable_variables))

train(model,n_epochs=5)

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# 13.2 TFRecord 포맷

크기가 다른 연속된 이진 레코드를 저장하는 단순한 이진포멧

In [ ]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
  f.write(b"this is the first record")
  f.write(b"and this is the second record")

In [ ]:
filepaths = ["my_data.tfrecord"]
datset =tf.data.TFRecordDataset(filepaths)
for item in datset:
  print(item)

### 13.2.1 압축된 TFRecord 파일

In [ ]:
#options 매개변수를 만들어서 압축가능.
options = tf.io.TFRecordOptions(compression_type = "GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
  f.write(b"This is the first record")
  f.write(b"And this is the second record")


In [ ]:
dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"],
                                  compression_type="GZIP")

for item in dataset:
  print(item)

### 13.2.2 프로토콜 버퍼 개요

각 숫자1,2,3은 필드식별자

In [ ]:
%%writefile person.proto
syntax = "proto3";
message Person {
  string name = 1;
  int32 id = 2;
  repeated string email = 3;
}



In [ ]:
!protoc person.proto --python_out=. --descriptor_set_out=person.desc --include_imports

In [ ]:
!ls person*

In [ ]:
from person_pb2 import Person

person = Person(name="Al", id=123, email=["a@b.com"])  # Person 생성
print(person)  # Person 출력

In [ ]:
person.name #필드읽기

In [ ]:
person.name = "Alice" #필드 수정하기

In [ ]:
person.email[0] #배열처럼 사용하는 반복필드

In [ ]:
person.email.append("c@d.com") #반복필드 추가

바이트 문자열로 직렬화

In [ ]:
s = person.SerializeToString()
s

직렬화한 바이트 문자열 파싱하기

In [ ]:
person2 =Person()
person2.ParseFromString(s)

In [ ]:
person == person2

tf.io.decode_proto()

(앞서 우리가 만든 것처럼) 사용자 정의 프로토콜 버퍼를 파싱해야 할때 사용

In [ ]:
person_tf = tf.io.decode_proto(
    bytes=s,
    message_type="Person",
    field_names=["name", "id", "email"],
    output_types=[tf.string, tf.int32, tf.string],
    descriptor_source="person.desc")

person_tf.values

### 12.2.3 텐서플로 프로토콜 버퍼

Feature은 byteslist, floatlist, int64중 하나를 담고있음.

Features는 특성이름과 특성값을 매핑한 딕셔너리를 가짐.

Example은 '하나'의 Feature 객체를 가짐

In [ ]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example


#Example 프로토콜 버퍼 만들기.
#샘플마다 하나의 example 프로토콜 버퍼
person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }))


In [ ]:
#Example 프로토콜 버퍼 직렬화 해서 TFRecord 파일로 저장하기
with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    f.write(person_example.SerializeToString())

### 13.2.3 Example 프로토콜 버퍼를 읽고 파싱하기

Example 프로토콜 버퍼를 읽기 위해

다시 tf.data.TFRecordDatset을 사용하여 불러오고

tf.io.parse_single_example(직렬화된 tfrecord, description)을 사용하여 각 example을 파싱

In [ ]:
feature_description = {
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""), #크기,타입,기본값을 표현한 FixedLenFeature
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0), #FixedLenFeature
    "emails": tf.io.VarLenFeature(tf.string), #길이가 가변적인 경우 특성의 타입만 표현한 tf.io.VarLenFeature
}
for serialized_example in tf.data.TFRecordDataset(["my_contacts.tfrecord"]):
    parsed_example = tf.io.parse_single_example(serialized_example, #직렬화된 tf record와
                                                feature_description) #설명 필요.

In [ ]:
parsed_example

In [ ]:
parsed_example["emails"].values[0]

가변길이특성은 희소텐서로 파싱됨. 

tf.sparse.to_dense: 희소텐서를 밀집텐서로 변환

In [ ]:
tf.sparse.to_dense(parsed_example["emails"], default_value=b"")

In [ ]:
parsed_example["emails"].values

### 13.2.4 SequenceExample 프로토콜 버퍼를 사용해 리스트의 리스트 다루기

문서는 문장의 리스트 : 리스트의 리스트

In [ ]:
from tensorflow.train import FeatureList, FeatureLists, SequenceExample

context = Features(feature={
    "author_id": Feature(int64_list=Int64List(value=[123])),
    "title": Feature(bytes_list=BytesList(value=[b"A", b"desert", b"place", b"."])),
    "pub_date": Feature(int64_list=Int64List(value=[1623, 12, 25]))
})

content = [["When", "shall", "we", "three", "meet", "again", "?"],
           ["In", "thunder", ",", "lightning", ",", "or", "in", "rain", "?"]]
comments = [["When", "the", "hurlyburly", "'s", "done", "."],
            ["When", "the", "battle", "'s", "lost", "and", "won", "."]]

def words_to_feature(words):
    return Feature(bytes_list=BytesList(value=[word.encode("utf-8")
                                               for word in words]))

content_features = [words_to_feature(sentence) for sentence in content]
comments_features = [words_to_feature(comment) for comment in comments]
            
#하나으 Feature객체와 한개 이상의 FeatureList를 가진 FeatureLists
sequence_example = SequenceExample(
    context=context,
    feature_lists=FeatureLists(feature_list={
        "content": FeatureList(feature=content_features),
        "comments": FeatureList(feature=comments_features)
    }))

In [ ]:
sequence_example

SequenceExample 직렬화

In [ ]:
serialized_sequence_example = sequence_example.SerializeToString()

SequenceExample 파싱

설명도 두개가 필요(1.Features, 2.FeatureList)

In [ ]:
#파싱을 위한 context 설명
context_feature_descriptions = {
    "author_id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "title": tf.io.VarLenFeature(tf.string),
    "pub_date": tf.io.FixedLenFeature([3], tf.int64, default_value=[0, 0, 0]),
}
#파싱을 위한 FeatureList 설명
sequence_feature_descriptions = {
    "content": tf.io.VarLenFeature(tf.string),
    "comments": tf.io.VarLenFeature(tf.string),
}

#하나의 SequenceExample 파싱: parse_single_sequence_example()
parsed_context, parsed_feature_lists = tf.io.parse_single_sequence_example(
    serialized_sequence_example, context_feature_descriptions,
    sequence_feature_descriptions)

In [ ]:
parsed_context

In [ ]:
parsed_context["title"].values

In [ ]:
parsed_feature_lists

가변길이 시퀀스를 갖고있는 특성리스트 래그드텐서로 바꾸기

In [ ]:
print(tf.RaggedTensor.from_sparse(parsed_feature_lists["content"]))

# 13.3 입력 특성 전처리

람다층

In [ ]:
means = np.mean(X_train, axis=0, keepdims=True)
stds = np.std(X_train, axis=0, keepdims = True)
eps = keras.backend.epsilon()
model = keras.models.Sequential([
                                 #람다층
          keras.layers.Lambda(lambda inputs: (input-means)/(std+eps))
])

사용자 정의 층

In [ ]:
class Standardization(keras.layers.Layer):
  def adapt(self,data_sample):
    self.means_ = np.mean(data_sample,axis=0,keepdims=True)
    self.stds_ = np.std(data_sample,axis=0,keepdims=True)
  def call(self,inputs):
    return (inputs-self.means_)/(self.stds_ + keras.backend.epsilon)

#층 추가 전 adapt로 계산
# std_layer = Standardization()
# std_layer.adapt(data_sample)

# 층 추가
# model= keras.Sequentail()
# model.add(std_layer)

### 13.3.1 원-핫 벡터를 사용해 범주형 특성 인코딩하기

In [ ]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
import pandas as pd

In [ ]:
csv_path = os.path.join(HOUSING_PATH, "housing.csv")
housing = pd.read_csv(csv_path)

In [ ]:
housing.head()

In [ ]:
vocab = ["<1H OCEAN","INLAND","NEAR OCEAN","NEAR BAY","ISLAND"]
#어휘 사전 정의
indices = tf.range(len(vocab), dtype = tf.int64)
#인덱스 텐서
table_init = tf.lookup.KeyValueTensorInitializer(vocab,indices)
#초기화 객체
num_oov_buckets = 2
#oov 를 위한 버켓. 충분히 안주면 같은 범주로 할당돼 충돌가능
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

In [ ]:
categories = tf.constant(["NEAR BAY","DESERT","INLAND","INLAND"])
cat_indices = table.lookup(categories) 
cat_indices

In [ ]:
cat_one_hot =tf.one_hot(cat_indices, depth = len(vocab)+ num_oov_buckets)
cat_one_hot

### 13.3.2 임베딩을 사용해 범주형 특성 인코딩하기

초기화

In [ ]:
embedding_dim =2
embed_init = tf.random.uniform([len(vocab)+ num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)

In [ ]:
embedding_matrix

In [ ]:
categories = tf.constant(["NEAR BAY","DESERT","INLAND","INLAND"])
cat_indices = table.lookup(categories)
cat_indices

In [ ]:
tf.nn.embedding_lookup(embedding_matrix,cat_indices)

In [ ]:
embedding = keras.layers.Embedding(input_dim =len(vocab)+num_oov_buckets,
                                   output_dim = embedding_dim)

In [ ]:
embedding(cat_indices)

원핫인코딩 , 임베딩을 이용하여 

 임베딩을 학습하는 케라스 모델

In [ ]:
regular_inputs = keras.layers.Input(shape=[8])
categories = keras.layers.Input(shape= [], dtype = tf.string)
#두개의 인풋

cat_indices = keras.layers.Lambda(lambda cats : table.lookup(cats))(categories)
#인덱스 찾기
cat_embed = keras.layers.Embedding(input_dim= 6, output_dim= 2)(cat_indices)
#임베딩에서 인덱스로 찾기
encoded_inputs = keras.layers.concatenate([regular_inputs,cat_embed])
outputs =keras.layers.Dense(1)(encoded_inputs)

model =keras.models.Model(inputs =[regular_inputs,categories],
                          outputs =[outputs])


### 13.3.3 케라스 전처리 층

keras.layers.Discretiztion : 구간을 나눠 원핫 인코딩 해줌

keras.layers.Normalization

keras.layers.TextVectorization: 인코딩,임베딩


**adapt() 후 일반적인 층처럼 이용 가능


PreprocessingStage 클래스를 사용해 연걸하여 파이프라인처럼 사용 가능

In [ ]:
# normalization = keras.layers.Normalization()
# discretization =keras.layers.Discretization([...])

# pipeline = keras.layers.PreprocessingStage([normalization,discretization])

# pipeline.adapt(data_sampe)

## 13.4 TF 변환

In [ ]:
try:
    import tensorflow_transform as tft

    def preprocess(inputs):  # inputs is a batch of input features
        median_age = inputs["housing_median_age"]
        ocean_proximity = inputs["ocean_proximity"]
        standardized_age = tft.scale_to_z_score(median_age - tft.mean(median_age))
        ocean_proximity_id = tft.compute_and_apply_vocabulary(ocean_proximity)
        return {
            "standardized_median_age": standardized_age,
            "ocean_proximity_id": ocean_proximity_id
        }
except ImportError:
    print("TF Transform is not installed. Try running: pip3 install -U tensorflow-transform")

# 13.5 텐서플로 데이터셋 (TFDS) 프로젝트

다양한 표준 데이터셋

In [ ]:
import tensorflow_datasets as tfds

datasets = tfds.load(name="mnist")
mnist_train, mnist_test = datasets["train"], datasets["test"]

In [ ]:
print(tfds.list_builders())

In [ ]:
plt.figure(figsize=(6,3))
mnist_train = mnist_train.repeat(5).batch(32).prefetch(1)
for item in mnist_train:
    images = item["image"]
    labels = item["label"]
    for index in range(5):
        plt.subplot(1, 5, index + 1)
        image = images[index, ..., 0]
        label = labels[index].numpy()
        plt.imshow(image, cmap="binary")
        plt.title(label)
        plt.axis("off")
    break

In [ ]:
dataset = tfds.load(name='mnist')

mnist_train,mnist_test = datasets['train'], datasets['test']
mnist_train = mnist_train.repeat(5).batch(32)

print(mnist_train.take(1))
#원래는 딕셔너리 형태
mnist_train =mnist_train.map(lambda items: (items["image"],items["label"]))
print(mnist_train.take(1))
#keras 는 (특성, 레이블) 튜플을 원하기 때문에 map으로 변환.
mnist_train =mnist_train.prefetch(1)

In [ ]:
for images,labels in mnist_train.take(1):
  print(images.shape)
  print(labels.numpy())

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
datasets = tfds.load(name="mnist", batch_size=32, as_supervised=True)
#as_supervised=True 지도학습을 위해 (특성,레이블) 튜플로 제공해줌

print(datasets)
#test와 train의 딕셔너리 형태
mnist_train = datasets["train"].prefetch(1)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28, 1]),
    keras.layers.Lambda(lambda images: tf.cast(images, tf.float32)),
    keras.layers.Dense(10, activation="softmax")])

In [ ]:
model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer= keras.optimizers.SGD(lr=1e-3),
              metrics =["accuracy"])
model.fit(mnist_train, steps_per_epoch =  60000//32, epochs=5)

# 연습문제 9

a.
_문제: (10장에서 소개한) 패션 MNIST 데이터셋을 적재하고 훈련 세트, 검증 세트, 테스트 세트로 나눕니다. 

훈련 세트를 섞은 다음 각 데이터셋을 TFRecord 파일로 저장합니다. 

각 레코드는 두 개의 특성을 가진 Example 프로토콜 버퍼, 즉 직렬화된 이미지(tf.io.serialize_tensor()를 사용해 이미지를 직렬화하세요)와 레이블입니다. 

참고: 용량이 큰 이미지일 경우 tf.io.encode_jpeg()를 사용할 수 있습니다. 많은 공간을 절약할 수 있지만 이미지 품질이 손해를 봅니다._

데이터 불러오기

In [ ]:
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()


X_train,X_valid = X_train_full[5000:],X_train_full[:5000]
y_train,y_valid = y_train_full[5000:],y_train_full[:5000]

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

훈련세트 섞고 TFRecord 파일로 저장하기

In [ ]:
train_set = tf.data.Dataset.from_tensor_slices((X_train,y_train))
train_set= train_set.shuffle(len(X_train))

valid_set = tf.data.Dataset.from_tensor_slices((X_valid,y_valid))
valid_set = valid_set.shuffle(len(X_valid))

test_set = tf.data.Dataset.from_tensor_slices((X_test,y_test))

Example 프로토콜 버퍼 생성

In [ ]:
def create_example(image,label):
  image_data = tf.io.serialize_tensor(image)
  return Example(
    features = Features(
      feature={
        "image":Feature(bytes_list = BytesList(value=[image_data.numpy()])),
        "label":Feature(int64_list =Int64List(value=[label]))
      }
    )
  )

In [ ]:
  for image, label in valid_set.take(1):
    print(create_example(image,label))

다음 함수는 주어진 데이터셋을 일련의 TFRecord 파일로 저장합니다. 이 예제는 라운드-로빈 방식으로 파일에 저장합니다. 이를 위해 dataset.enumerate() 메서드로 모든 샘플을 순회하고 저장할 파일을 겨정하기 위해 index % n_shards를 계산합니다. 표준 contextlib.ExitStack 클래스를 사용해 쓰는 동안 I/O 에러의 발생 여부에 상관없이 모든 writer가 적절히 종료되었는지 확인합니다.

In [ ]:
from contextlib import ExitStack

def write_tfrecords(name, dataset, n_shards=10):
    paths = ["{}.tfrecord-{:05d}-of-{:05d}".format(name, index, n_shards)
             for index in range(n_shards)]
    with ExitStack() as stack:
        writers = [stack.enter_context(tf.io.TFRecordWriter(path))
                   for path in paths]
        for index, (image, label) in dataset.enumerate():
            shard = index % n_shards
            example = create_example(image, label)
            writers[shard].write(example.SerializeToString())
    return paths

In [ ]:
train_filepaths = write_tfrecords("my_fashion_mnist.train", train_set)
valid_filepaths = write_tfrecords("my_fashion_mnist.valid", valid_set)
test_filepaths = write_tfrecords("my_fashion_mnist.test", test_set)

b.
문제: tf.data로 각 세트를 위한 효율적인 데이터셋을 만듭니다. 

마지막으로 이 데이터셋으로 입력 특성을 표준화하는 전처리 층을 포함한 케라스 모델을 훈련합니다. 

텐서보드로 프로파일 데이터를 시각화하여 가능한 한 입력 파이프라인을 효율적으로 만들어보세요.

map 위한 파싱 함수

In [ ]:
def preprocess(tfrecord):
  feature_descriptions ={
        "image" : tf.io.FixedLenFeature([], tf.string,default_value=""),
        "label" : tf.io.FixedLenFeature([],tf.int64, default_value=-1)
  } 

  example = tf.io.parse_single_example(tfrecord,feature_descriptions)
  image =tf.io.parse_tensor(example["image"], out_type = tf.uint8)

  image = tf.reshape(image,shape=[28,28])
  return image, example["label"]


전처리 함수(데이터 적재와 전처리 합치기)

TFRecord 포멧과 tf.data api 사용

In [ ]:
def mnist_dataset(filepaths, n_read_threads=5, shuffle_buffer_size=None,
                  n_parse_threads=5, batch_size=32, cache=True):
  dataset = tf.data.TFRecordDataset(filepaths,
                                    num_parallel_reads=n_read_threads)
  if cache:
    dataset = dataset.cache()
  if shuffle_buffer_size:
    dataset =dataset.shuffle(shuffle_buffer_size)
  dataset =dataset.map(preprocess,num_parallel_calls=n_parse_threads)
  dataset = dataset.batch(batch_size)

  return dataset.prefetch(1)

In [ ]:
train_set = mnist_dataset(train_filepaths, shuffle_buffer_size=60000)
valid_set = mnist_dataset(valid_filepaths)
test_set = mnist_dataset(test_filepaths)

In [ ]:
train_set

In [ ]:
for X,y in train_set.take(1):
  for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(X[i].numpy(), cmap='binary')
    plt.axis("off")
    plt.title(str(y[i].numpy()))

입력 특성을 표준화하는 전처리 층

--> 사용자 정의 층 방법 :모델에 포함하는 방법. 


In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
class Standardiztion(keras.layers.Layer):
  def adapt(self,data_sample):
      self.means_ = np.mean(data_sample,axis=0, keepdims=True)
      self.stds_ = np.std(data_sample,axis=0,keepdims=True)

  def call(self, inputs):
    return (inputs-self.means_)/(self.stds_+keras.backend.epsilon())

standardization = Standardiztion(input_shape=[28,28])

sample_image_batches = train_set.take(100).map(lambda image, label: image)
sample_images = np.concatenate(list(sample_image_batches.as_numpy_iterator()),
                               axis=0).astype(np.float32)

standardization.adapt(sample_images)

model = keras.models.Sequential([
                                standardization,
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(100,activation='relu'),
                                 keras.layers.Dense(10,activation='softmax')
                                
])

model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer = keras.optimizers.Nadam(),metrics="accuracy")



텐서보드를 위한 콜백, 학습

In [ ]:
from datetime import datetime
logs =os.path.join(os.curdir, "my_logs",
                   "run_"+datetime.now().strftime("%Y%m%d_%H%M%S"))

tensorboard_cb = keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch=10
)

model.fit(train_set,epochs=5,validation_data=valid_set,
          callbacks=[tensorboard_cb])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

## 연습문제 10 
(어려워서 옮겨놓은 후 분석)

문제: 이 연습문제에서 데이터셋을 다운로드 및 분할하고 tf.data.Dataset 객체를 만들어 데이터를 적재하고 효율적으로 전처리하겠습니다. 그다음 Embedding 층을 포함한 이진 분류 모델을 만들고 훈련시킵니다.

a.
문제: 인터넷 영화 데이터베이스의 영화 리뷰 50,000개를 담은 영화 리뷰 데이터셋을 다운로드합니다. 

이 데이터는 train과 test라는 두 개의 디렉터리로 구성되어 있습니다. 

각 디렉터리에는 12,500개의 긍정 리뷰를 담은 pos 서브디렉터리와 12,500개의 부정 리뷰를 담은 neg 서브디렉터리가 있습니다. 

리뷰는 각각 별도의 텍스트 파일에 저장되어 있습니다. (전처리된 BOW를 포함해) 다른 파일과 디렉터리가 있지만 이 연습문제에서는 무시합니다.

In [ ]:
from pathlib import Path

DOWNLOAD_ROOT = "http://ai.stanford.edu/~amaas/data/sentiment/"
FILENAME = "aclImdb_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)
#압축 풀기 전 '/root/.keras/datasets/aclImdb_v1.tar.gz'
#압축 푼 후 /root/.keras/datasets/aclImdb
path = Path(filepath).parent / "aclImdb"
path

In [ ]:
#aclImdb 파일 구조를 시각화

for name, subdirs, files in os.walk(path):
  #os.walk(path) : path의 폴더 트리구조 반환. 상위부터 하위까지.
    indent = len(Path(name).parts) - len(path.parts)
    print("    " * indent + Path(name).parts[-1] + os.sep)
    for index, filename in enumerate(sorted(files)):
        if index == 3:
            print("    " * (indent + 1) + "...")
            break
        print("    " * (indent + 1) + filename)

In [ ]:
def review_paths(dirpath):
    return [str(path) for path in dirpath.glob("*.txt")]
    #glob : *.txt로 끝나는 것들 다 반환

train_pos = review_paths(path / "train" / "pos") #path/train/pos 의 .txt 다 반환
train_neg = review_paths(path / "train" / "neg")
test_valid_pos = review_paths(path / "test" / "pos")
test_valid_neg = review_paths(path / "test" / "neg")

len(train_pos), len(train_neg), len(test_valid_pos), len(test_valid_neg)

b.
문제: 테스트 세트를 검증 세트(15,000개)와 테스트 세트(10,000개)로 나눕니다.

In [ ]:
np.random.shuffle(test_valid_pos)
#섞음.
test_pos = test_valid_pos[:5000]
test_neg = test_valid_neg[:5000]
valid_pos = test_valid_pos[5000:]
valid_neg = test_valid_neg[5000:]

In [ ]:
test_pos[:10]

In [ ]:
with open("/root/.keras/datasets/aclImdb/test/pos/2502_8.txt") as review_file:
        a=review_file.read()
print(type(a))
print(a)
#read()는 스트링 반환

c.
문제: tf.data를 사용해 각 세트에 대한 효율적인 데이터셋을 만듭니다.

In [ ]:
def imdb_dataset(filepaths_positive, filepaths_negative):
  reviews=[]
  labels=[]

  for filepaths, label in ((filepaths_negative,0), (filepaths_positive,1)):#레이블 부여
    for filepath in filepaths:
      with open(filepath) as review_file:
        reviews.append(review_file.read())
        #하나씩 파일 열어서 리스트에 str을 append
      labels.append(label)

  #tf.data api로 변환.
  return tf.data.Dataset.from_tensor_slices(
      (tf.constant(reviews), tf.constant(labels))
  )


In [ ]:
#경로를 각각 받아서, 레이블 주고 tf.data api로 변경
for X,y in imdb_dataset(train_pos,train_neg).take(3): 
  print(X)
  print(y)
  print()

In [ ]:
#tf.data api를 사용하여 데이터를 전처리.
batch_size = 32

train_set = imdb_dataset(train_pos, train_neg).shuffle(25000).batch(batch_size).prefetch(1)
valid_set = imdb_dataset(valid_pos, valid_neg).batch(batch_size).prefetch(1)
test_set = imdb_dataset(test_pos, test_neg).batch(batch_size).prefetch(1)

d.
_문제: 리뷰를 전처리하기 위해 TextVectorization 층을 사용한 이진 분류 모델을 만드세요.

 TextVectorization 층을 아직 사용할 수 없다면 (또는 도전을 좋아한다면) 사용자 전처리 층을 만들어보세요. 
 
 tf.strings 패키지에 있는 함수를 사용할 수 있습니다. 예를 들어 lower()로 소문자로 만들거나 regex_replace()로 구두점을 공백으로 바꾸고 split()로 공백을 기준으로 단어를 나눌 수 있습니다. 
 
 룩업 테이블을 사용해 단어 인덱스를 출력하세요. adapt() 메서드로 미리 층을 적응시켜야 합니다._




먼저 리뷰를 전처리하는 함수를 만듭니다. 이 함수는 리뷰를 300자로 자르고 소문자로 변환합니다. 그다음 br /와 글자가 아닌 모든 문자를 공백으로 바꾸고 리뷰를 단어로 분할해 마지막으로 각 리뷰가 n_words 개수의 토큰이 되도록 패딩하거나 잘라냅니다:

In [ ]:
def preprocess(X_batch, n_words=50):
    shape = tf.shape(X_batch) * tf.constant([1, 0]) + tf.constant([0, n_words])
    Z = tf.strings.substr(X_batch, 0, 300)
    #300자 서브스트링으로 자름
    Z = tf.strings.lower(Z)
    #소문자
    Z = tf.strings.regex_replace(Z, b"<br\\s*/?>", b" ")
    #br 공백으로
    Z = tf.strings.regex_replace(Z, b"[^a-z]", b" ")
    #글자아닌거 공백으로
    Z = tf.strings.split(Z)
    #띄어쓰기 단어 스플릿
    return Z.to_tensor(shape=shape, default_value=b"<pad>")
    #텐서로


X_example = tf.constant(["It's a great, great movie! I loved it.", "It was terrible, run away!!!"])
preprocess(X_example)

### 사용자 정의층 말고 layers.experiments.preprocessing.TextVectorization 사용해보기

In [ ]:
print(train_set)

#레이블 떼기 (textvectorization은 X만 필요)
train_example = train_set.map(lambda x,y:x)

for i in train_example.take(1):
  print(i)


In [ ]:
from tensorflow import keras


TextVectorization_layer = keras.layers.experimental.preprocessing.TextVectorization(output_mode= "count")
TextVectorization_layer.adapt(train_example.take(1))

inputs = keras.layers.Input(shape=[],dtype=tf.string)
cat_indices =TextVectorization_layer(inputs)                        
layer = keras.layers.Dense(100, activation="relu")(cat_indices)
output = keras.layers.Dense(1, activation="sigmoid")(layer)

model = keras.models.Model(inputs=[inputs],
                           outputs =[output])
    

model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, epochs=5, validation_data=valid_set)

In [ ]:
 from collections import Counter

def get_vocabulary(data_sample, max_size=1000):
    preprocessed_reviews = preprocess(data_sample).numpy()
    counter = Counter()
    for words in preprocessed_reviews:
        for word in words:
            if word != b"<pad>":
                counter[word] += 1
    return [b"<pad>"] + [word for word, count in counter.most_common(max_size)]

get_vocabulary(X_example)

In [ ]:
class TextVectorization(keras.layers.Layer):
    def __init__(self, max_vocabulary_size=1000, n_oov_buckets=100, dtype=tf.string, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.max_vocabulary_size = max_vocabulary_size
        self.n_oov_buckets = n_oov_buckets

    def adapt(self, data_sample):
        self.vocab = get_vocabulary(data_sample, self.max_vocabulary_size)
        words = tf.constant(self.vocab)
        word_ids = tf.range(len(self.vocab), dtype=tf.int64)
        vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
        self.table = tf.lookup.StaticVocabularyTable(vocab_init, self.n_oov_buckets)
        
    def call(self, inputs):
        preprocessed_inputs = preprocess(inputs)
        return self.table.lookup(preprocessed_inputs)

In [ ]:
text_vectorization = TextVectorization()

text_vectorization.adapt(X_example)
text_vectorization(X_example)

In [ ]:
max_vocabulary_size = 1000
n_oov_buckets = 100

sample_review_batches = train_set.map(lambda review, label: review)
sample_reviews = np.concatenate(list(sample_review_batches.as_numpy_iterator()),
                                axis=0)

text_vectorization = TextVectorization(max_vocabulary_size, n_oov_buckets,
                                       input_shape=[])
text_vectorization.adapt(sample_reviews)

In [ ]:
text_vectorization(X_example)

In [ ]:
text_vectorization.vocab[:10]

In [ ]:
simple_example = tf.constant([[1, 3, 1, 0, 0], [2, 2, 0, 0, 0]])
tf.reduce_sum(tf.one_hot(simple_example, 4), axis=1)

In [ ]:
class BagOfWords(keras.layers.Layer):
    def __init__(self, n_tokens, dtype=tf.int32, **kwargs):
        super().__init__(dtype=tf.int32, **kwargs)
        self.n_tokens = n_tokens
    def call(self, inputs):
        one_hot = tf.one_hot(inputs, self.n_tokens)
        return tf.reduce_sum(one_hot, axis=1)[:, 1:]

In [ ]:
bag_of_words = BagOfWords(n_tokens=4)
bag_of_words(simple_example)

In [ ]:
n_tokens = max_vocabulary_size + n_oov_buckets + 1 # add 1 for <pad>
bag_of_words = BagOfWords(n_tokens)

In [ ]:
model = keras.models.Sequential([
                                
    text_vectorization,
    bag_of_words,
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid"),
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, epochs=5, validation_data=valid_set)

e.
문제: Embedding 층을 추가하고 단어 개수의 제곱근을 곱하여 리뷰마다 평균 임베딩을 계산하세요(16장 참조). 이제 스케일이 조정된 이 평균 임베딩을 모델의 다음 부분으로 전달할 수 있습니다.
각 리뷰의 평균 임베딩을 계산하고 리뷰의 단어 개수의 제곱근을 곱하기 위해 간단한 함수를 정의합니다:


In [ ]:
def compute_mean_embedding(inputs):
    not_pad = tf.math.count_nonzero(inputs, axis=-1)
    n_words = tf.math.count_nonzero(not_pad, axis=-1, keepdims=True)    
    sqrt_n_words = tf.math.sqrt(tf.cast(n_words, tf.float32))
    return tf.reduce_mean(inputs, axis=1) * sqrt_n_words

another_example = tf.constant([[[1., 2., 3.], [4., 5., 0.], [0., 0., 0.]],
                               [[6., 0., 0.], [0., 0., 0.], [0., 0., 0.]]])
compute_mean_embedding(another_example)

In [ ]:
tf.reduce_mean(another_example, axis=1) * tf.sqrt([[2.], [1.]])

In [ ]:
embedding_size = 20

model = keras.models.Sequential([
    text_vectorization,
    keras.layers.Embedding(input_dim=n_tokens,
                           output_dim=embedding_size,
                           mask_zero=True), # <pad> tokens => zero vectors
    keras.layers.Lambda(compute_mean_embedding),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid"),
])

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
model.fit(train_set, epochs=5, validation_data=valid_set)

In [ ]:
import tensorflow_datasets as tfds

datasets = tfds.load(name="imdb_reviews")
train_set, test_set = datasets["train"], datasets["test"]

In [ ]:
for example in train_set.take(1):
    print(example["text"])
    print(example["label"])